In [1]:
import pandas as pd
from pathlib import Path
import sys

import pandas as pd

sys.path.append(r'C:\Users\User\OneDrive\Desktop\Code\msba_edgar')
from ut_msba_edgar_scraper import Downloader
from ut_msba_edgar_scraper.msba_utils import get_ratings_df, stock_mapping_df
ratings_df = get_ratings_df() # this is our scraping universe

In [4]:
def read_master_log():
    df = pd.read_csv('logs/master_log.csv')
    df['gvkey'] = df['gvkey'].astype(str)
    return df.iloc[:,1:].copy()

df_master_log = read_master_log()
df_successes = df_master_log[df_master_log.success == True]
df_successes.shape

(95420, 16)

In [2]:
df_angel = pd.read_csv('data/fallen.csv')
df_angel

,gvkey,splticrm,datadate,cik,fic,gsector,loc,tic,Year,Sub_IG,Sub_IG_p,splticrm_p,Fallen
0,1004,BB-,2003-04-30,1750.0,USA,20.0,USA,AIR,2003,1.0,0.0,BBB-,1.0
1,1045,BB,2001-09-30,6201.0,USA,20.0,USA,AAL,2001,1.0,0.0,BBB-,1.0
2,1048,BB,2002-12-31,65695.0,USA,10.0,USA,4267A,2002,1.0,0.0,BBB,1.0
3,1164,BB,2002-05-31,723527.0,USA,50.0,USA,MCIP,2002,1.0,0.0,BBB,1.0
4,1230,BB+,2016-12-31,766421.0,USA,20.0,USA,ALK,2016,1.0,0.0,BBB-,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,174022,BB+,2006-12-31,1398987.0,USA,60.0,USA,RLGY,2006,1.0,0.0,BBB,1.0
425,176928,BB,2008-04-30,1383312.0,USA,45.0,USA,BR,2008,1.0,0.0,BBB-,1.0
426,177745,D,2011-10-31,1401106.0,USA,40.0,USA,MFGLQ,2011,1.0,0.0,BBB-,1.0
427,178814,BB+,2017-01-31,1403256.0,USA,40.0,USA,SCU,2017,1.0,0.0,BBB-,1.0


In [13]:
angel_keys = df_angel.gvkey.unique().astype(str)
truths = [key in df_successes.gvkey.values for key in angel_keys]
sum(truths)

In [42]:
stock_mapping_df.gvkey.nunique()

17259

In [31]:
gvkeys = ratings_df['gvkey'].unique()
print(f'unique gvkeys = {len(gvkeys)}')

unique gvkeys = 3414


In [28]:
gvkeys_ari = ['1661','154358']
'1661' in  ratings_df.gvkey.values # This shows that it didn't show up because it wasn't in our universe

False

In [30]:
# ratings_df[ratings_df.gvkey == '154358']

In [36]:
df_successes.gvkey.nunique()

2106

In [37]:
df_successes.gvkey.nunique()/len(gvkeys) # 62 % of universe

0.616871704745167

In [34]:
df_successes.groupby('gvkey').accession_number.nunique().mean()

45.30864197530864

In [35]:
df_successes.groupby('gvkey').accession_number.nunique().median()

41.0

In [38]:
mapping_gvkeys = stock_mapping_df.gvkey.unique()
mapping_ratings = ratings_df[ratings_df.gvkey.isin(mapping_gvkeys)]

In [40]:
mapping_ratings

,gvkey,splticrm,datadate,cik,fic,gsector,loc,tic
0,1004,BBB,2000-01-31,1750,USA,20.0,USA,AIR
1,1004,BBB,2000-02-29,1750,USA,20.0,USA,AIR
2,1004,BBB,2000-03-31,1750,USA,20.0,USA,AIR
3,1004,BBB,2000-04-30,1750,USA,20.0,USA,AIR
4,1004,BBB,2000-05-31,1750,USA,20.0,USA,AIR
...,...,...,...,...,...,...,...,...
329181,311524,B+,2014-07-31,1547063,USA,15.0,USA,TAM
329182,311524,B+,2014-08-31,1547063,USA,15.0,USA,TAM
329183,311524,B+,2014-09-30,1547063,USA,15.0,USA,TAM
329184,311524,B+,2014-10-31,1547063,USA,15.0,USA,TAM


In [4]:
# Combine with Compustat
merged_df = df_master_log.merge(ratings_df.drop_duplicates(subset='gvkey'),on='gvkey')
merged_df.shape

In [7]:
sectors = merged_df.groupby('gsector')

In [8]:
ratings_df.groupby('gsector').gvkey.nunique()

gsector
10.0    331
15.0    243
20.0    450
25.0    630
30.0    177
35.0    263
40.0    445
45.0    301
50.0    185
55.0    280
60.0    109
Name: gvkey, dtype: int64

In [9]:
sectors.gvkey.nunique()

gsector
10.0    206
15.0    153
20.0    303
25.0    388
30.0    104
35.0    157
40.0    240
45.0    201
50.0    132
55.0    170
60.0     62
Name: gvkey, dtype: int64

In [10]:
(sectors.gvkey.nunique() / ratings_df.groupby('gsector').gvkey.nunique()) # Average is 62%

gsector
10.0    0.622356
15.0    0.629630
20.0    0.673333
25.0    0.615873
30.0    0.587571
35.0    0.596958
40.0    0.539326
45.0    0.667774
50.0    0.713514
55.0    0.607143
60.0    0.568807
Name: gvkey, dtype: float64

In [41]:
# (sectors.gvkey.nunique() / ratings_df.groupby('gsector').gvkey.nunique()).plot(kind='bar')